In [5]:
import facebook
import pandas as pd
import pymongo
from pymongo import MongoClient
connection = MongoClient("mongodb://localhost:27017")
db = connection.IndeedJobPostDB
df = pd.DataFrame(list(db.Job_Posts.find()))
df

,_id,Title,Job_Type,Company,Salary,Schedule,Location,Date_Posted,Current_Date,Summary,Apply_link
0,647483c0b4ccc7f531eab330,Host,,Chili's,Estimated $22.9K - $29K a year,,"Little Falls, NJ 07424",Just posted,23-05-29,Flexible part-time or full-time schedule. Comm...,https://www.indeed.com//rc/clk?jk=1f9dd7702acd...
1,647483c0b4ccc7f531eab331,Server,,Chili's,Estimated $19.2K - $24.3K a year,,"Staten Island, NY 10314 (Richmondtown area)",Just posted,23-05-29,Flexible part-time or full-time schedule. Prov...,https://www.indeed.com//rc/clk?jk=702b98a418b1...
2,647483c0b4ccc7f531eab332,Line Cook,,Chili's,$15.00 - $20.25 an hour,,"Glendale, NY 11385",Just posted,23-05-29,Keep food orders flowing continuously from the...,https://www.indeed.com//rc/clk?jk=c4ef85c7a19a...
3,647483c0b4ccc7f531eab333,Restaurant Busser,,AOI Japanese Restaurant,From $14.50 an hour,,"Iselin, NJ 08830",Just posted,23-05-29,"As a busser, you will be responsible for clean...",https://www.indeed.com//rc/clk?jk=7b3c4c12a72d...
4,647483c0b4ccc7f531eab334,Assistant Food and Beverage Outlets Manager,,APA Hotel Woodbridge,"$45,000 - $55,000 a year",,"Iselin, NJ 08830",Just posted,23-05-29,Manage inventory ensuring outlets are fully st...,https://www.indeed.com//company/TKP-Bar-&-Gril...
...,...,...,...,...,...,...,...,...,...,...,...
925,647483c0b4ccc7f531eab6cd,Team Member,,"036823,1347 KENNEDY BLVD,,BAYONNE,NJ",Estimated $29.8K - $37.7K a year,,"Bayonne, NJ 07002",Posted 5 days ago,23-05-29,"As a Pizza Hut team member, you can be the smi...",https://www.indeed.com//rc/clk?jk=e5d8f6888673...
926,647483c0b4ccc7f531eab6ce,Line Cook,,DoubleTree by Hilton Newark Penn Station,Estimated $33.1K - $41.9K a year,,"Newark, NJ 07102 (Central Business District area)",Posted 5 days ago,23-05-29,"Attached to Newark Penn Station, The Doubletre...",https://www.indeed.com//rc/clk?jk=efae4767cff2...
927,647483c0b4ccc7f531eab6cf,Team Member,,"036817,180 MAIN ST,,WEST ORANGE,NJ",Estimated $29.8K - $37.7K a year,,"West Orange, NJ 07052",Posted 5 days ago,23-05-29,"As a Pizza Hut team member, you can be the smi...",https://www.indeed.com//rc/clk?jk=c0f14b638ab6...
928,647483c0b4ccc7f531eab6d0,Line Cook,,Park Avenue Club,$20 - $25 an hour,,"Florham Park, NJ",EmployerActive 2 days ago,23-05-29,Immediately informing a manager or supervisor ...,https://www.indeed.com//company/Park-Avenue-Cl...


In [ ]:
groups = ['658314902622656']
msg = "Testing Automation: Post 02"
access_token = "EAAJHKZAZAZBPbsBALZAv8UJCV5LZA9cHZCkAqjfBB6YZAOjQW5ZC89Jj0evnMTPycx4s5mlQ9ZBx7jaDenZAgziyvJRgZAtQyF4J3r1Ip2rYZAxOW7DCZCzAIkIMpvj6ZBVxmNuCSXRlchlYaZCjPdF74rSBc9ehlbvvd9KCWLwXIntmEAvnfQPon8qsFt31Yiz6ZCzZC04bK2cY6ietAdOQpBBUdm9ytFjSdHc2nOi7wZBnghFjvwZAJIGjKt7YsZAP"
graph = facebook.GraphAPI(access_token=access_token)

for group in groups:
    x = graph.put_object(group, 'feed', message=msg)
    print(x)
    print("Posted Successfully!")